In [1]:
from GUI import GUI

In [2]:
gui = GUI(img_file='data/input/7.jpg')

In [3]:
gui.element_detection(True, True, True)
gui.visualize_element_detection()

[Text Detection Completed in 2.740 s] Input: data/input/7.jpg Output: data/output\ocr\7.json
[Compo Detection Completed in 0.717 s] Input: data/input/7.jpg Output: data/output\ip\7.json
[Merge Completed] Input: data/input/7.jpg Output: data/output\uied\7.jpg


In [4]:
gui.layout_recognition()
gui.visualize_layout_recognition()

[Layout Recognition Completed in 1.867 s] Input: data/input/7.jpg Output: data/output\layout\7.json
Sun Aug 29 09:49:49 2021 




In [7]:
gui.visualize_layout_recognition()